# Segmenting and Clustering Neighborhoods in Toronto
### Import Libraries

In [30]:
from bs4 import BeautifulSoup
import requests
import urllib
import urllib.request
import os
import pandas as pd
import csv
import numpy as np

### Create function

In [5]:
def make_soup(url):
    thepage = urllib.request.urlopen(url)
    soupdata = BeautifulSoup(thepage,"html.parser")
    return soupdata

### Request Data

In [6]:
#source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = make_soup('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

### Find Data
Find rows and columns, Create header, Create csv file

In [7]:
postcode=postcodesaved=""
table = soup.find_all('table')[0] # Grab the first table

for record in table.findAll('tr'):
    postcode=""
    for data in record.findAll('td'):
        postcode=postcode+","+data.text
    postcodesaved = postcodesaved + "\n" + postcode[1:]

header="Postcode,Borough,Neighbourhood"   
file=open(os.path.expanduser('PostCodes.csv'),"wb")
file.write(bytes(header,encoding='ascii',errors='ignore'))
file.write(bytes(postcodesaved,encoding='ascii',errors='ignore'))

9063

### Read csv file show Dataframe

In [97]:
df=pd.read_csv('PostCodes.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is Not assigned

In [91]:
df2=df[df.Borough != 'Not assigned']
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Change Neighbourhood 'Not assigned' by Borough name

In [101]:
df3=df2.replace('Not assigned', df2.Borough)
df3.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Confirming there is no more 'Not assigned'

In [122]:
df3.loc[df3['Neighbourhood']== 'Not assigned']

,Postcode,Borough,Neighbourhood


###  Combine two rows into one row with the neighborhoods separated with a comma

In [119]:
df5=df2.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(','.join)
df5

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### Do a check to verify count of Neighbourhoods per Postcode
Selecting a random Postcodes from below and compare the quantities from above

In [121]:
df2.groupby('Postcode').nunique()

,Postcode,Borough,Neighbourhood
Postcode,,,
M1B,1,1,2
M1C,1,1,3
M1E,1,1,3
M1G,1,1,1
M1H,1,1,1
M1J,1,1,1
M1K,1,1,3
M1L,1,1,3
M1M,1,1,3


In [123]:
df5.shape

(103, 3)